In [8]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from keras_vggface.vggface import VGGFace

from mtcnn import MTCNN

import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import re
from sklearn.model_selection import train_test_split

In [2]:
face_detector = MTCNN()
print("Face detector model loded...")

Face detector model loded...


In [5]:
# Lists to store images and labels
train_images = []
train_labels = []
label_map = {}  # Mapping of labels to indices
label_index = 0

def add_image(image_path):
    """Loads images and extracts embeddings for training."""
    global label_index

    root, _ = os.path.splitext(image_path)
    label = os.path.split(root)[-1]
    match = re.search(r"^(\w+)_", label)
    label = match.group(1)

    if label not in label_map:
        label_map[label] = label_index
        label_index += 1

    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    faces = face_detector.detect_faces(image)
    
    if not faces:
        print(f"No face found in `{label}`, skipping.")
        return

    if len(faces) > 1:
        print(f"Multiple faces found in `{label}`, skipping.")
        return
    x, y, w, h = faces[0]["box"]
    x1, y1 = max(0, x), max(0, y)
    x2, y2 = min(image.shape[1], x + w), min(image.shape[0], y + h)

    cropped_face = image[y1:y2, x1:x2]
    resized_face = cv2.resize(cropped_face, (224, 224))
    resized_face = resized_face.astype("float32") / 255.0  # Normalize

    train_images.append(resized_face)
    train_labels.append(label_map[label])

# Load images from dataset
for dir, _, files in os.walk("../Member Photos"):
    for file in files:
        add_image(os.path.join(dir, file))

train_images = np.array(train_images)
train_labels = np.array(train_labels)
train_labels = to_categorical(train_labels, num_classes=len(label_map))

print(f"✅ Loaded {len(train_images)} images for training.")

2/2 [==============================] - 0s 4ms/step
Multiple faces found in `ethan`, skipping.
2/2 [==============================] - 0s 13ms/step
Multiple faces found in `eunice`, skipping.
2/2 [==============================] - 0s 8ms/step
Multiple faces found in `eunice`, skipping.
1/1 [==============================] - 0s 19ms/step
Multiple faces found in `eunice`, skipping.
1/1 [==============================] - 0s 24ms/step
Multiple faces found in `eunice`, skipping.
1/1 [==============================] - 0s 18ms/step
Multiple faces found in `jinwei`, skipping.
1/1 [==============================] - 0s 18ms/step
Multiple faces found in `junyong`, skipping.
1/1 [==============================] - 0s 19ms/step
Multiple faces found in `junyong`, skipping.
1/1 [==============================] - 0s 17ms/step
Multiple faces found in `junyong`, skipping.
1/1 [==============================] - 0s 17ms/step
✅ Loaded 93 images for training.


In [11]:
BASE = r"C:/Users/jy158/Desktop/NTU/Notes/Y4S2/EE4228 Intelligent System Design/Assignment/real-time-one-shot-face-recognition/"
checkpoint_path = BASE + "checkpoints/resnet50_face_recognition.h5"

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, stratify=train_labels, random_state=42
)

# Create a callback that saves the model's weights during training
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_best_only=True,  # Save only the best model based on validation loss
    monitor="val_loss",
    mode="min",
    verbose=1
)

# Load the pre-trained ResNet50 model without the top classification layer
base_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))

# Add custom layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)  # Feature embedding layer
x = Dense(6, activation="softmax")(x)  # Classification layer (adjust num_classes)

# Define new model
model = Model(inputs=base_model.input, outputs=x)

# Freeze the pre-trained layers (optional)
for layer in base_model.layers:
    layer.trainable = False  # Freeze weights

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

# Train the model
# Train the model with checkpointing enabled
model.fit(
    train_images, train_labels,
    batch_size=32,
    epochs=200,
    validation_data=(val_images, val_labels),
    callbacks=[checkpoint_callback]  # Add the callback here
)

Epoch 1/200
2/2 [==============================] - ETA: 0s - loss: 1.8723 - accuracy: 0.2128
Epoch 1: val_loss improved from inf to 1.75681, saving model to C:/Users/jy158/Desktop/NTU/Notes/Y4S2/EE4228 Intelligent System Design/Assignment/real-time-one-shot-face-recognition/checkpoints\resnet50_face_recognition.h5
2/2 [==============================] - 6s 2s/step - loss: 1.8723 - accuracy: 0.2128 - val_loss: 1.7568 - val_accuracy: 0.2500
Epoch 2/200
2/2 [==============================] - ETA: 0s - loss: 1.8061 - accuracy: 0.2128
Epoch 2: val_loss improved from 1.75681 to 1.73593, saving model to C:/Users/jy158/Desktop/NTU/Notes/Y4S2/EE4228 Intelligent System Design/Assignment/real-time-one-shot-face-recognition/checkpoints\resnet50_face_recognition.h5
2/2 [==============================] - 2s 945ms/step - loss: 1.8061 - accuracy: 0.2128 - val_loss: 1.7359 - val_accuracy: 0.2500
Epoch 3/200
2/2 [==============================] - ETA: 0s - loss: 1.7875 - accuracy: 0.1489
Epoch 3: val_los